In [0]:
!pip install scattertext
!pip install flask_cors
!pip install python-decouple
!pip install -U spacy
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 6.8MB 2.5MB/s 
  Created wheel for python-decouple: filename=python_decouple-3.3-cp36-none-any.whl size=9024 sha256=e77a6d92116b9ad5c3e9bcdaaa1e828abe7b71a333349cb45f5584fab4fdea2c
  Stored in directory: /root/.cache/pip/wheels/f4/9b/79/5eec9ea205382552b014e749cb8a2b4ce0d6836a6f4db5ac0c
Successfully built python-decouple
     |████████████████████████████████| 10.4MB 2.8MB/s 
     |████████████████████████████████| 2.2MB 32.7MB/s 
     |████████████████████████████████| 122kB 55.0MB/s 
     |████████████████████████████████| 3.7MB 20.8MB/s 
  Found existing installation: preshed 2.0.1
    Uninstalling preshed-2.0.1:
      Successfully uninstalled preshed-2.0.1
  Found existing installation: blis 0.2.4
    Uninstalling blis-0.2.4:
      Successfully uninstalled blis-0.2.4
  Found existing installation: thinc 7.0.8
    Uninstalling thinc-7.0.8:
      Successfully uninstalled thinc-7.0.8
  Found existing installation: spacy 2.1.9
    Uninstalling sp

In [0]:
from flask import Flask, render_template, request, jsonify
import json
import warnings
import pandas as pd
import spacy
import scattertext as st
from lxml import html
from requests import Session
from concurrent.futures import ThreadPoolExecutor as Executor
import requests
# from flask_cors import CORS
# from decouple import config
import re
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.max_colwidth', 1000)
nlp = spacy.load("en_core_web_sm")#if you run into problems here, 'Restart Runtime' and run all, it might fix things.
base_url = "https://www.yelp.com/biz/" 
api_url = "/review_feed?sort_by=date_desc&start="
bid = 'Rc1lxc5lSKJYd162JHNMfQ'

class Scraper():
    def __init__(self):
        self.data = pd.DataFrame()

    def get_data(self, n, bid=bid):
        with Session() as s:
            with s.get(base_url+bid+api_url+str(n*20)) as resp: #makes an http get request to given url and returns response as json
                r = json.loads(resp.content) #converts json response into a dictionary
                _html = html.fromstring(r['review_list']) #loads from dictionary

                dates = _html.xpath("//div[@class='review-content']/descendant::span[@class='rating-qualifier']/text()")
                reviews = [el.text for el in _html.xpath("//div[@class='review-content']/p")]
                ratings = _html.xpath("//div[@class='review-content']/descendant::div[@class='biz-rating__stars']/div/@title")

                df = pd.DataFrame([dates, reviews, ratings]).T

                self.data = pd.concat([self.data,df])

    def scrape(self): #makes it faster
        # multithreaded looping
        with Executor(max_workers=40) as e:
            list(e.map(self.get_data, range(10)))

s = Scraper()
s.scrape()
df = s.data


def customtokensize(text):
    return re.findall("[\w']+", str(text))

df['tokenized_text'] = df[1].apply(customtokensize)
stopwords = ['and','was','were','had','check-in','=','= =','u','want', 'u want', 'cuz','him',"i've",'on', 'her','told','ins', '1 check','I', 'i"m', 'i', ' ', 'it', "it's", 'it.','they', 'the', 'this','its', 'l','they','this',"don't",'the ', ' the', 'it', 'i"ve', 'i"m', '!', '1','2','3','4', '5','6','7','8','9','0','/','.',',']

def filter_stopwords(text):
  nonstopwords = []
  for i in text:
    if i not in stopwords:
      nonstopwords.append(i)
  return nonstopwords
df['tokenized_text'] = df['tokenized_text'].apply(filter_stopwords)
df['parts_of_speech_reference'] = df['tokenized_text'].apply(filter_stopwords)
df['parts_of_speech_reference'] = df['parts_of_speech_reference'].str.join(' ')

In [0]:
def find_phrases(x):
  """Create a list where adjectives come immediately before nouns for each review"""
  long_phrase_list = []
  doc = nlp(str(x))
  for token in range(len(doc)):
    sub_list = []
    try:
      if (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token].pos_ == 'VERB' and doc[token+1].pos_ == 'NOUN') or (doc[token-1].pos_ == 'NOUN' and doc[token].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'ADJ'and doc[token+2].pos_ =='NOUN')or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='VERB'and doc[token+2].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'NOUN'and doc[token+2].pos_ =='NOUN'):
        if doc[token-1] not in sub_list and doc[token] not in sub_list and doc[token+1] not in sub_list and doc[token+2] not in sub_list and doc[token+3] not in sub_list:
          sub_list.append(doc[token-1])
          sub_list.append(doc[token])
          sub_list.append(doc[token+1])
          sub_list.append(doc[token+2])
          sub_list.append(doc[token+3])
    except IndexError as e:
      pass
    try:
      if (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token].pos_ == 'VERB' and doc[token+1].pos_ == 'NOUN') or (doc[token-1].pos_ == 'NOUN' and doc[token].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'ADJ'and doc[token+2].pos_ =='NOUN')or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='VERB'and doc[token+2].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'NOUN'and doc[token+2].pos_ =='NOUN'):
        if doc[token-1] not in sub_list and doc[token] not in sub_list and doc[token+1] not in sub_list and doc[token+2] not in sub_list:
          sub_list.append(doc[token-1])
          sub_list.append(doc[token])
          sub_list.append(doc[token+1])
          sub_list.append(doc[token+2])
    except IndexError as e:
      pass
    try:
      if (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token].pos_ == 'VERB' and doc[token+1].pos_ == 'NOUN') or (doc[token-1].pos_ == 'NOUN' and doc[token].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'ADJ'and doc[token+2].pos_ =='NOUN')or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='VERB'and doc[token+2].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'NOUN'and doc[token+2].pos_ =='NOUN'):
        if doc[token-1] not in sub_list and doc[token] not in sub_list and doc[token+1] not in sub_list:
          sub_list.append(doc[token-1])
          sub_list.append(doc[token])
          sub_list.append(doc[token+1])
    except IndexError as e:
      pass
    try:
      if (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token].pos_ == 'VERB' and doc[token+1].pos_ == 'NOUN') or (doc[token-1].pos_ == 'NOUN' and doc[token].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'ADJ'and doc[token+2].pos_ =='NOUN')or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='VERB'and doc[token+2].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'NOUN'and doc[token+2].pos_ =='NOUN'):
        if doc[token] not in sub_list and doc[token+1] not in sub_list:
          sub_list.append(doc[token])
          sub_list.append(doc[token+1])
    except IndexError as e:
      pass
    try:
      #captures words at the end of reviews
      if (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token-1].pos_ == 'NOUN' and doc[token].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token-1].pos_ == 'ADJ'and doc[token].pos_ =='NOUN') or (doc[token-1].pos_ == 'VERB'and doc[token].pos_ =='NOUN')or (doc[token-1].pos_ == 'NOUN'and doc[token].pos_ =='NOUN'):
        if doc[token] not in sub_list and doc[token+1] not in sub_list:
          sub_list.append(doc[token-1])
          sub_list.append(doc[token])
    except IndexError as e:
      pass
    try:
      #captures words from the start of reviews
      if (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token].pos_ == 'NOUN' and doc[token+1].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN')or (doc[token].pos_ == 'NOUN'and doc[token+1].pos_ =='NOUN'):
        if doc[token] not in sub_list and doc[token+1] not in sub_list:
          sub_list.append(doc[token])
          sub_list.append(doc[token+1])
    except IndexError as e:
      pass
    if len(sub_list) != 0:
      long_phrase_list.append(sub_list)
  return long_phrase_list


df['long_phrase_list'] = df['parts_of_speech_reference'].apply(find_phrases)
df.sample(20)

,0,1,2,tokenized_text,parts_of_speech_reference,long_phrase_list
8,\n 7/6/2019\n,"I took my parents to this restaurant on the first day of their vacation visiting me in the city. They loved this restaurant: great ambience, quick and attentive service, impressive entertainment with 2 different films projected on opposing walls, and excellent food. My father's a picky eater but loved the lamb chops with zucchini. I ordered a salad with salmon and it was superb. I look forward to trying other items on the menu",5.0 star rating,"[took, my, parents, to, restaurant, first, day, of, their, vacation, visiting, me, in, city, They, loved, restaurant, great, ambience, quick, attentive, service, impressive, entertainment, with, different, films, projected, opposing, walls, excellent, food, My, father's, a, picky, eater, but, loved, lamb, chops, with, zucchini, ordered, a, salad, with, salmon, superb, look, forward, to, trying, other, items, menu]",took my parents to restaurant first day of their vacation visiting me in city They loved restaurant great ambience quick attentive service impressive entertainment with different films projected opposing walls excellent food My father's a picky eater but loved lamb chops with zucchini ordered a salad with salmon superb look forward to trying other items menu,"[[menu, took, my, parents, to], [restaurant, first, day, of, their], [first, day], [vacation, visiting], [vacation, visiting, me, in, city], [They, loved, restaurant, great, ambience], [loved, restaurant], [restaurant, great, ambience, quick, attentive], [great, ambience], [ambience, quick, attentive, service, impressive], [quick, attentive, service, impressive, entertainment], [attentive, service, impressive, entertainment, with], [service, impressive, entertainment, with, different], [impressive, entertainment], [with, different, films, projected, opposing], [different, films], [films, projected, opposing, walls, excellent], [projected, opposing, walls, excellent, food], [opposing, walls], [walls, excellent, food, My, father], [excellent, food], [a, picky, eater, but, loved], [picky, eater], [eater, but, loved, lamb, chops], [zucchini, ordered], [zucchini, ordered, a, salad, with], [trying, other, items, menu], [other, items]]"
14,\n 8/3/2019\n,"Wonderful place for endless brunch on the weekend. Good food, attentive service, lively atmosphere, and love the old film and TV clips playing on the walls!",5.0 star rating,"[Wonderful, place, for, endless, brunch, weekend, Good, food, attentive, service, lively, atmosphere, love, old, film, TV, clips, playing, walls]",Wonderful place for endless brunch weekend Good food attentive service lively atmosphere love old film TV clips playing walls,"[[walls, Wonderful, place, for, endless], [Wonderful, place], [for, endless, brunch, weekend, Good], [endless, brunch], [brunch, weekend], [weekend, Good, food, attentive, service], [Good, food], [food, attentive], [attentive, service, lively, atmosphere, love], [service, lively, atmosphere, love, old], [lively, atmosphere], [atmosphere, love, old, film, TV], [love, old, film, TV, clips], [old, film], [film, TV], [TV, clips], [clips, playing, walls]]"
12,\n 7/2/2019\n,"I ordered Spicy lamb meatballs,so tasty,the cheesecake paired with wine,and for last sparkling rose,employees are awesome very attentive,I will definitely going back,thanks to everybody.",5.0 star rating,"[ordered, Spicy, lamb, meatballs, so, tasty, cheesecake, paired, with, wine, for, last, sparkling, rose, employees, are, awesome, very, attentive, will, definitely, going, back, thanks, to, everybody]",ordered Spicy lamb meatballs so tasty cheesecake paired with wine for last sparkling rose employees are awesome very attentive will definitely going back thanks to everybody,"[[everybody, ordered, Spicy, lamb, meatballs], [ordered, Spicy, lamb, meatballs, so], [Spicy, lamb], [lamb, meatballs], [so, tasty, cheesecake, paired, with], [tasty, cheesecake], [cheesecake, paired, with, wine, for], [f

In [0]:
def find_two_word_phrases(x):
  """Create a list where adjectives come immediately before nouns for each review"""
  two_word_phrases = []
  doc = nlp(str(x))
  for token in range(len(doc)):
    try:
      sub_list = []
      if (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token].pos_ == 'VERB' and doc[token+1].pos_ == 'NOUN') or (doc[token-1].pos_ == 'NOUN' and doc[token].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'ADJ'and doc[token+2].pos_ =='NOUN')or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='VERB'and doc[token+2].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'NOUN'and doc[token+2].pos_ =='NOUN'):
        if doc[token-1] not in sub_list and doc[token] not in sub_list and doc[token+1] not in sub_list and doc[token+2] not in sub_list and doc[token+3] not in sub_list:
          sub_list.append(doc[token-1])
          sub_list.append(doc[token])
          sub_list.append(doc[token+1])
          sub_list.append(doc[token+2])
          sub_list.append(doc[token+3])
    except IndexError as e:
      pass
    try:
      sub_list = []
      if (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token].pos_ == 'VERB' and doc[token+1].pos_ == 'NOUN') or (doc[token-1].pos_ == 'NOUN' and doc[token].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'ADJ'and doc[token+2].pos_ =='NOUN')or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='VERB'and doc[token+2].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'NOUN'and doc[token+2].pos_ =='NOUN'):
        if doc[token-1] not in sub_list and doc[token] not in sub_list and doc[token+1] not in sub_list and doc[token+2] not in sub_list:
          sub_list.append(doc[token-1])
          sub_list.append(doc[token])
          sub_list.append(doc[token+1])
          sub_list.append(doc[token+2])
    except IndexError as e:
      pass
    try:
      sub_list = []
      if (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token].pos_ == 'VERB' and doc[token+1].pos_ == 'NOUN') or (doc[token-1].pos_ == 'NOUN' and doc[token].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'ADJ'and doc[token+2].pos_ =='NOUN')or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='VERB'and doc[token+2].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'NOUN'and doc[token+2].pos_ =='NOUN'):
        if doc[token-1] not in sub_list and doc[token] not in sub_list and doc[token+1] not in sub_list:
          sub_list.append(doc[token-1])
          sub_list.append(doc[token])
          sub_list.append(doc[token+1])
    except IndexError as e:
      pass
    try:
      sub_list = []
      if (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token].pos_ == 'VERB' and doc[token+1].pos_ == 'NOUN') or (doc[token-1].pos_ == 'NOUN' and doc[token].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'ADJ'and doc[token+2].pos_ =='NOUN')or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='VERB'and doc[token+2].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'NOUN'and doc[token+2].pos_ =='NOUN'):
        if doc[token] not in sub_list and doc[token+1] not in sub_list:
          sub_list.append(doc[token])
          sub_list.append(doc[token+1])
    except IndexError as e:
      pass
    try:
      sub_list = []
      #captures words at the end of reviews
      if (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token-1].pos_ == 'NOUN' and doc[token].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token-1].pos_ == 'ADJ'and doc[token].pos_ =='NOUN') or (doc[token-1].pos_ == 'VERB'and doc[token].pos_ =='NOUN')or (doc[token-1].pos_ == 'NOUN'and doc[token].pos_ =='NOUN'):
        if doc[token] not in sub_list and doc[token+1] not in sub_list:
          sub_list.append(doc[token-1])
          sub_list.append(doc[token])
    except IndexError as e:
      pass
    try:
      sub_list = []
      #captures words from the start of reviews
      if (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token].pos_ == 'NOUN' and doc[token+1].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN')or (doc[token].pos_ == 'NOUN'and doc[token+1].pos_ =='NOUN'):
        if doc[token] not in sub_list and doc[token+1] not in sub_list:
          sub_list.append(doc[token])
          sub_list.append(doc[token+1])
    except IndexError as e:
      pass
    if len(sub_list) != 0:
      two_word_phrases.append(sub_list)
  return two_word_phrases


df['two_word_phrases'] = df['parts_of_speech_reference'].apply(find_two_word_phrases)
df.sample(20)

,0,1,2,tokenized_text,parts_of_speech_reference,long_phrase_list,two_word_phrases
11,\n 11/20/2019\n,"Service was very poor, we had what seemed like more than three waiters serve us. As kind as they were, I found myself waiting more than eating. The Tommy's Pie was also not my personal favorite, I'd rather go for the gnocchi which was pretty good and the cheese board which was fantastic!",3.0 star rating,"[Service, very, poor, we, what, seemed, like, more, than, three, waiters, serve, us, As, kind, as, found, myself, waiting, more, than, eating, The, Tommy's, Pie, also, not, my, personal, favorite, I'd, rather, go, for, gnocchi, which, pretty, good, cheese, board, which, fantastic]",Service very poor we what seemed like more than three waiters serve us As kind as found myself waiting more than eating The Tommy's Pie also not my personal favorite I'd rather go for gnocchi which pretty good cheese board which fantastic,"[[fantastic, Service, very, poor, we], [waiters, serve], [waiters, serve, us, As, kind], [myself, waiting, more, than, eating], [also, not, my, personal, favorite], [pretty, good, cheese, board, which], [good, cheese], [cheese, board]]","[[Service, very], [waiters, serve], [waiting, more], [not, my], [good, cheese], [cheese, board]]"
17,\n 10/2/2019\n,"The rooftop is great, prices are pretty good. Wouldn't go out of my way to come back but it's convenient",3.0 star rating,"[The, rooftop, is, great, prices, are, pretty, good, Wouldn't, go, out, of, my, way, to, come, back, but, convenient]",The rooftop is great prices are pretty good Wouldn't go out of my way to come back but convenient,"[[is, great, prices, are, pretty], [great, prices], [back, but, convenient]]","[[great, prices], [but, convenient]]"
13,\n 11/19/2019\n,"Amazing!! Loved the breakfast brunch prefix option! Tons of yummy options! Started using off with this amazing honey drizzled ricotta with toasted baguette slices - We tried the lamb meatballs with tatziki/pita and kale caeser for appetizer, brussel sprouts with candied bacon and honey yogurt with fruit for sharing, and for ""main event"" bea burger with fries and eggs benedict! All for $50!",5.0 star rating,"[Amazing, Loved, breakfast, brunch, prefix, option, Tons, of, yummy, options, Started, using, off, with, amazing, honey, drizzled, ricotta, with, toasted, baguette, slices, We, tried, lamb, meatballs, with, tatziki, pita, kale, caeser, for, appetizer, brussel, sprouts, with, candied, bacon, honey, yogurt, with, fruit, for, sharing, for, main, event, bea, burger, with, fries, eggs, benedict, All, for, 50]",Amazing Loved breakfast brunch prefix option Tons of yummy options Started using off with amazing honey drizzled ricotta with toasted baguette slices We tried lamb meatballs with tatziki pita kale caeser for appetizer brussel sprouts with candied bacon honey yogurt with fruit for sharing for main event bea burger with fries eggs benedict All for 50,"[[50, Amazing, Loved, breakfast, brunch], [Amazing, Loved, breakfast, brunch, prefix], [Loved, breakfast], [breakfast, brunch], [brunch, prefix], [prefix, option], [option, Tons], [of, yummy, options, Started, using], [yummy, options], [options, Started, using, off, with], [with, amazing, honey, drizzled, ricotta], [amazing, honey], [honey, drizzled, ricotta, with, toasted], [drizzled, ricotta], [with, toasted, baguette, slices, We], [toasted, baguette], [baguette, slices], [We, tried, lamb, meatballs, with], [tried, lamb], [lamb, meatballs], [honey, yogurt], [honey, yogurt], [for, main, event, bea, burger], [main, event], [fries, eggs], [fries, eggs], [eggs, benedict, All, for, 50]]","[[Loved, breakfast], [breakfast, brunch], [brunch, prefix], [prefix, option], [option, Tons], [yummy, options], [options, Started], [amazing, honey], [honey, drizzled], [drizzled, ricotta], [toasted, baguette], [baguette, slices], [tried, lamb], [lamb, meatballs], [honey, yogurt], [main, event], [fries, eggs], [eggs, benedict]]"
15,\n 8/20/2019\n,Sea bass and the 

In [0]:
phrase_count = df[['long_phrase_list','two_word_phrases', 2]]
phrase_count.sample(2)

,long_phrase_list,two_word_phrases,2
18,"[[good, Best, cocktails, ever, Highly], [Best, cocktails], [date, night], [date, night], [Burborn, based, drink, is, fantastic], [based, drink], [is, fantastic, dinner, menu, pasta], [fantastic, dinner], [dinner, menu]]","[[Best, cocktails], [date, night], [based, drink], [fantastic, dinner], [dinner, menu]]",5.0 star rating
11,"[[customers, seem], [customers, seem, to, always, have], [hygiene, rating], [hygiene, rating], [door, Nasty, scary, place], [Nasty, scary, place]]","[[customers, seem], [hygiene, rating], [scary, place]]",1.0 star rating


In [0]:
import numpy as np

s= phrase_count.apply(lambda x: pd.Series(x['two_word_phrases']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'two_word_phrases'

phrase_count = phrase_count.drop('two_word_phrases', axis=1).join(s)
phrase_count.head(2)

,long_phrase_list,2,two_word_phrases
0,"[[sure, Food], [Food, portion], [price, ratio], [price, ratio], [With, main, entrees, all, at], [main, entrees], [n't, surprised, size, would, be], [surprised, size], [size, would, be, small, Some], [however, not, worth, that, price], [price, baked], [price, baked, scrambled, eggs, one], [baked, scrambled, eggs, one, of], [scrambled, eggs], [A, few, pieces, of, mozzarella], [few, pieces], [of, mozzarella, potatoes, plus, a], [mozzarella, potatoes], [a, small, serving, size, of], [small, serving, size, of, eggs], [serving, size], [eggs, not, worth, that, price]]",3.0 star rating,"[Food, portion]"
0,"[[sure, Food], [Food, portion], [price, ratio], [price, ratio], [With, main, entrees, all, at], [main, entrees], [n't, surprised, size, would, be], [surprised, size], [size, would, be, small, Some], [however, not, worth, that, price], [price, baked], [price, baked, scrambled, eggs, one], [baked, scrambled, eggs, one, of], [scrambled, eggs], [A, few, pieces, of, mozzarella], [few, pieces], [of, mozzarella, potatoes, plus, a], [mozzarella, potatoes], [a, small, serving, size, of], [small, serving, size, of, eggs], [serving, size], [eggs, not, worth, that, price]]",3.0 star rating,"[price, ratio]"


In [0]:
phrase_count['two_word_phrases'] = phrase_count['two_word_phrases'].astype(str)
phrase_count['two_word_phrases'] = [''.join(map(str, l)) for l in phrase_count['two_word_phrases']]
# phrase_count['long_phrase_list'] = phrase_count['long_phrase_list'].apply(lambda x: ''.join(map(str, x)))
# phrase_count['long_phrase_list'] = phrase_count['long_phrase_list'].replace(',', '')
phrase_count.head(2)
# pd.pivot_table(phrase_count, index=['long_phrase_list'], columns=[2], aggfunc=np.sum)

,long_phrase_list,2,two_word_phrases
0,"[[sure, Food], [Food, portion], [price, ratio], [price, ratio], [With, main, entrees, all, at], [main, entrees], [n't, surprised, size, would, be], [surprised, size], [size, would, be, small, Some], [however, not, worth, that, price], [price, baked], [price, baked, scrambled, eggs, one], [baked, scrambled, eggs, one, of], [scrambled, eggs], [A, few, pieces, of, mozzarella], [few, pieces], [of, mozzarella, potatoes, plus, a], [mozzarella, potatoes], [a, small, serving, size, of], [small, serving, size, of, eggs], [serving, size], [eggs, not, worth, that, price]]",3.0 star rating,"[Food, portion]"
0,"[[sure, Food], [Food, portion], [price, ratio], [price, ratio], [With, main, entrees, all, at], [main, entrees], [n't, surprised, size, would, be], [surprised, size], [size, would, be, small, Some], [however, not, worth, that, price], [price, baked], [price, baked, scrambled, eggs, one], [baked, scrambled, eggs, one, of], [scrambled, eggs], [A, few, pieces, of, mozzarella], [few, pieces], [of, mozzarella, potatoes, plus, a], [mozzarella, potatoes], [a, small, serving, size, of], [small, serving, size, of, eggs], [serving, size], [eggs, not, worth, that, price]]",3.0 star rating,"[price, ratio]"


In [0]:
phrase_count.shape

(15743, 3)

In [0]:
phrase_count_pivot = pd.pivot_table(phrase_count, index=['two_word_phrases'], columns=[2], aggfunc='count')
phrase_count_pivot.head(30)

long_phrase_list                                  \
2                       1.0 star rating 2.0 star rating 3.0 star rating   
two_word_phrases                                                          
[21st, birthday]                    4.0             3.0             4.0   
[43rd, 9th]                         NaN             NaN             2.0   
[5th, anniversary]                  NaN             NaN             1.0   
[Amazing, cocktails]                NaN             1.0             1.0   
[American, classics]                NaN             1.0             2.0   
[American, restaurant]              1.0             NaN             3.0   
[Asian, flavor]                     NaN             NaN             NaN   
[Atmosphere, fun]                   NaN             1.0             2.0   
[B, switch]                         1.0             NaN             2.0   
[Bad, food]                         2.0             1.0             2.0   
[Beautiful, bar]                    NaN             3.0             NaN   
[Best, cocktails]                   3.0             1.0             3.0   
[Best, part]                        NaN             3.0             NaN   
[Best, place]                       NaN             NaN             3.0   
[Booked, restaurant]                NaN             1.0             1.0   
[Cheap, wine]                       1.0             1.0             1.0   
[Cool, atmosphere]                  NaN             3.0             NaN   
[Cool, vibe]                        NaN             NaN             1.0   
[Cute, place]                       NaN             NaN             1.0   
[Cute, restaurant]                  NaN             NaN             1.0   
[DECISION, EVER]                    NaN             NaN             1.0   
[Decent, food]                      NaN             1.0             2.0   
[Delicious, food]                   1.0             NaN             2.0   
[Delicious, tapas]                  NaN             NaN             1.0   
[Delightful, place]                 NaN             1.0             1.0   
[Drinks, fancy]                     NaN             NaN             2.0   
[Drinks, fun]                       NaN             1.0             1.0   
[Drinks, star]                      1.0             1.0             1.0   
[Duck, turnovers]                   NaN             1.0             1.0   
[Excellent, cocktail]               1.0             NaN             1.0   

                                                        
2                      4.0 star rating 5.0 star rating  
two_word_phrases                                        
[21st, birthday]                   4.0             5.0  
[43rd, 9th]                        3.0             5.0  
[5th, anniversary]                 3.0             6.0  
[Amazing, cocktails]               2.0             6.0  
[American, classics]               1.0             6.0  
[American, restaurant]             2.0             4.0  
[Asian, flavor]                    3.0             7.0  
[Atmosphere, fun]                  1.0             6.0  
[B, switch]                        3.0             4.0  
[Bad, food]                        2.0             3.0  
[Beautiful, bar]                   3.0             4.0  
[Best, cocktails]                  7.0             6.0  
[Best, part]                       3.0             4.0  
[Best, place]                      2.0             5.0  
[Booked, restaurant]               3.0             5.0  
[Cheap, wine]                      3.0             4.0  
[Cool, atmosphere]                 3.0             4.0  
[Cool, vibe]                       4.0             5.0  
[Cute, place]                      4.0             5.0  
[Cute, restaurant]                 3.0             6.0  
[DECISION, EVER]                   4.0             5.0  
[Decent, food]                     1.0             6.0  
[Delicious, food]                  3.0             4.0  
[Delicious, tapas]                 3.0             6.0  
[Delight

In [0]:
phrase_count_pivot_sorted.sort_values(phrase_count_pivot_sorted.columns.tolist()).sort_index(level=1, ascending=False, 
                                                        sort_remaining=False)
phrase_count_pivot_sorted.tail(30)

two_word_phrases                                  \
2                     1.0 star rating 2.0 star rating 3.0 star rating   
long_phrase_list                                                        
[bottomless, drinks]              2.0             1.0             3.0   
[food, cocktails]                 NaN             NaN             4.0   
[recommend, place]                NaN             1.0             8.0   
[date, spot]                      1.0             NaN             3.0   
[pre, theater]                    1.0             2.0             2.0   
[little, place]                   NaN             NaN             4.0   
[date, night]                     4.0             3.0             8.0   
[girls, night]                    2.0             NaN             8.0   
[Loved, place]                    NaN             1.0             2.0   
[mushroom, pizza]                 1.0             2.0             7.0   
[dining, area]                    NaN             NaN             8.0   
[bar, type]                       NaN             8.0             2.0   
[pork, dumplings]                 1.0             3.0             4.0   
[broadway, show]                  2.0             2.0             8.0   
[drink, menu]                     NaN             4.0             4.0   
[food, drinks]                    2.0             1.0             6.0   
[family, style]                   NaN             NaN             4.0   
[bottomless, brunch]              4.0             8.0             5.0   
[Great, place]                    NaN             3.0             5.0   
[sea, bass]                       NaN             4.0             7.0   
[pear, salad]                     NaN             9.0             4.0   
[duck, turnover]                  2.0             3.0             3.0   
[steak, eggs]                     1.0             4.0             7.0   
[Great, food]                     1.0             3.0             6.0   
[green, pasta]                    2.0             4.0             6.0   
[tomato, soup]                    NaN             2.0             8.0   
[great, food]                     3.0             3.0             9.0   
[beet, salad]                     NaN             8.0             8.0   
[lamb, meatballs]                 3.0             3.0            12.0   
[theater, district]               NaN             2.0             5.0   

                                                      
2                    4.0 star rating 5.0 star rating  
long_phrase_list                                      
[bottomless, drinks]             9.0            15.0  
[food, cocktails]               11.0            15.0  
[recommend, place]               5.0            16.0  
[date, spot]                    10.0            16.0  
[pre, theater]                   9.0            16.0  
[little, place]                  9.0            17.0  
[date, night]                    8.0            17.0  
[girls, night]                  12.0            18.0  
[Loved, place]                   9.0            18.0  
[mushroom, pizza]               11.0            19.0  
[dining, area]                  12.0            20.0  
[bar, type]                     10.0            20.0  
[pork, dumplings]               12.0            20.0  
[broadway, show]                 8.0            20.0  
[drink, menu]                   12.0            20.0  
[food, drinks]                  11.0            20.0  
[family, style]                 16.0            20.0  
[bottomless, brunch]            13.0            20.0  
[Great, place]                  11.0            21.0  
[sea, bass]                      8.0            21.0  
[pear, salad]                   13.0            24.0  
[duck, turnover]                15.0            27.0  
[steak, eggs]                   11.0            27.0  
[Great, food]                   12.0            28.0  
[green, pasta]                  19.0            29.0  
[tomato, soup]                  20.0            30.0  
[great, food]   

In [0]:
phrase_count_pivot_sorted_low = phrase_count_pivot.sort_values(by=('long_phrase_list','1.0 star rating'), ascending=False)

phrase_count_pivot_sorted_low.head(30)

long_phrase_list                                  \
2                          1.0 star rating 2.0 star rating 3.0 star rating   
two_word_phrases                                                             
[Great, atmosphere]                    5.0             3.0             7.0   
[slow, service]                        5.0             2.0             5.0   
[21st, birthday]                       4.0             3.0             4.0   
[bottomless, brunch]                   4.0             8.0             5.0   
[good, food]                           4.0             5.0             6.0   
[service, is]                          4.0             4.0            12.0   
[not, sure]                            3.0             4.0             4.0   
[lamb, meatballs]                      3.0             4.0            13.0   
[great, food]                          3.0             3.0             9.0   
[great, place]                         3.0             1.0             4.0   
[Best, cocktails]                      3.0             1.0             3.0   
[Made, reservations]                   3.0             2.0             5.0   
[delicious, ambiance]                  2.0             2.0             2.0   
[green, noodles]                       2.0             1.0             2.0   
[frequent, spot]                       2.0             2.0             2.0   
[front, door]                          2.0             1.0             2.0   
[fun, atmosphere]                      2.0             1.0             3.0   
[fun, spot]                            2.0             1.0             2.0   
[garden, area]                         2.0             1.0             2.0   
[decent, wine]                         2.0             1.0             2.0   
[green, pasta]                         2.0             4.0             6.0   
[food, plenty]                         2.0             2.0             2.0   
[ham, bacon]                           2.0             1.0             2.0   
[hot, meals]                           2.0             1.0             2.0   
[know, check]                          2.0             2.0             2.0   
[last, year]                           2.0             2.0             2.0   
[late, hours]                          2.0             2.0             2.0   
[least, stars]                         2.0             1.0             2.0   
[date, night]                          2.0             2.0             6.0   
[local, restos]                        2.0             1.0             2.0   
[local, spots]                         2.0             2.0             2.0   
[food, service]                        2.0             4.0             9.0   
[food, drink]                          2.0             NaN             4.0   
[food, drinks]                         2.0             3.0             7.0   
[delicious, food]                      2.0             2.0             2.0   
[delicious, portion]                   2.0             1.0             2.0   
[dessert, bring]                       2.0             2.0             2.0   
[dessert, coming]                      2.0             2.0             2.0   
[dinner, menu]                         2.0             1.0             2.0   
[downside, would]                      2.0             1.0             2.0   
[drinks, tasted]                       2.0             2.0             2.0   
[early, bird]                          2.0             1.0             2.0   
[deeeeflee, recommend]                 2.0             2.0             2.0   
[eggs, benny]                          2.0             1.0             2.0   
[eggs, put]                            2.0             2.0             2.0   
[enjoyed, meatballs]                   2.0             1.0             2.0   
[enough, ham]                          2.0             1.0             2.0   
[fair, price]                          2.0             1.0             2.0   
[fantastic, dinner]                    2.0      

In [0]:
phrase_count_pivot_sorted_low = phrase_count_pivot.sort_values(by=('long_phrase_list','2.0 star rating'), ascending=False)

phrase_count_pivot_sorted_low.head(30)

long_phrase_list                                  \
2                       1.0 star rating 2.0 star rating 3.0 star rating   
two_word_phrases                                                          
[bottomless, brunch]                4.0             8.0             5.0   
[but, not]                          1.0             6.0             9.0   
[ordered, a]                        1.0             5.0             9.0   
[good, food]                        4.0             5.0             6.0   
[wait, staff]                       2.0             5.0             4.0   
[pear, salad]                       NaN             5.0             3.0   
[lamb, meatballs]                   3.0             4.0            13.0   
[bar, type]                         NaN             4.0             1.0   
[loved, restaurant]                 NaN             4.0             2.0   
[love, projectors]                  NaN             4.0             1.0   
[not, great]                        NaN             4.0             2.0   
[green, pasta]                      2.0             4.0             6.0   
[service, is]                       4.0             4.0            12.0   
[not, sure]                         3.0             4.0             4.0   
[food, service]                     2.0             4.0             9.0   
[beet, salad]                       NaN             4.0             4.0   
[hits, spot]                        NaN             3.0             NaN   
[husband, daughter]                 NaN             3.0             NaN   
[ordered, cocktails]                NaN             3.0             NaN   
[great, food]                       3.0             3.0             9.0   
[great, meal]                       NaN             3.0             NaN   
[intimate, restaurant]              NaN             3.0             NaN   
[ordered, The]                      NaN             3.0             NaN   
[chocolate, lava]                   NaN             3.0             NaN   
[great, restaurant]                 NaN             3.0             NaN   
[great, table]                      NaN             3.0             NaN   
[cappuccino, topped]                NaN             3.0             NaN   
[cake, thought]                     NaN             3.0             NaN   
[brunch, policy]                    NaN             3.0             NaN   
[cake, cappuccino]                  NaN             3.0             NaN   

                                                        
2                      4.0 star rating 5.0 star rating  
two_word_phrases                                        
[bottomless, brunch]              13.0            20.0  
[but, not]                        10.0            24.0  
[ordered, a]                      13.0            22.0  
[good, food]                      10.0            15.0  
[wait, staff]                      8.0            11.0  
[pear, salad]                      8.0            14.0  
[lamb, meatballs]                 22.0            38.0  
[bar, type]                        5.0            10.0  
[loved, restaurant]                6.0             8.0  
[love, projectors]                 5.0            10.0  
[not, great]                       4.0            10.0  
[green, pasta]                    19.0            29.0  
[service, is]                     11.0            19.0  
[not, sure]                       10.0            19.0  
[food, service]                   14.0            21.0  
[beet, salad]                     12.0            20.0  
[hits, spot]                       3.0             4.0  
[husband, daughter]                3.0             4.0  
[ordered, cocktails]               3.0             4.0  
[great, food]                     15.0            30.0  
[great, meal]                      3.0             4.0  
[intimate, restaurant]             3.0             4.0  
[ordered, The]                     6.0            11.0  
[chocolate, lava]                  6.0            11.0  
[great, 